## Imports

In [1]:
import sys
import pandas as pd
from pathlib import Path
sys.path.append("../")
from spacy.tokens import DocBin, Doc
from typing import List, Tuple
from lib.utils import tei2spacy, nlp_model_fr, sample_files, print_corpus_summary, SalientSentenceSelector

In [2]:
SPACY_CORPUS_SERIALIZED_PATH = "./data/corpus.spacy"

## Loading pre-processed data

In [3]:
if Path(SPACY_CORPUS_SERIALIZED_PATH).exists():
    spacy_corpus = DocBin(store_user_data=True).from_disk(SPACY_CORPUS_SERIALIZED_PATH)
    print(f"Loaded serialize spacy corpus from {SPACY_CORPUS_SERIALIZED_PATH}")
    print_corpus_summary(spacy_corpus, nlp_model_fr)
else:
    spacy_corpus = DocBin(store_user_data=True)

Loaded serialize spacy corpus from ./data/corpus.spacy
Number of documents in the corpus: 12
Number of entities in the corpus: 3204
Number of tokens in the corpus: 184314


In [4]:
docs = spacy_corpus.get_docs(nlp_model_fr.vocab)

## Adding entity linking information to docs in corpus

In [5]:
entity_fishing_pipe = nlp_model_fr.add_pipe(
    "entityfishing", config={
        "api_ef_base": "http://nerd.huma-num.fr/nerd/service"
    }
)

In [6]:
#nlp_model_fr.pipe_names

In [7]:
el_docs = [
    entity_fishing_pipe(spacy_doc)
    for spacy_doc in docs
]

## Selection of salient sentences

In [8]:
# take a sample document    
sample_doc = el_docs[9]

In [9]:
sass = SalientSentenceSelector(sample_doc)

Document bpt6k5458862p.tar.gz.tei_segmented_ner.xml contains 460 PER entities; 134 linked and 326 non-linked
Document bpt6k5458862p.tar.gz.tei_segmented_ner.xml contains 49 PER unique entities
Document bpt6k5458862p.tar.gz.tei_segmented_ner.xml contains 11 LOC entities; 9 linked and 2 non-linked
Document bpt6k5458862p.tar.gz.tei_segmented_ner.xml contains 7 LOC unique entities


In [21]:
sass.place_entities

{'Q894836': Entity(qid='Q894836', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['PARIS'], short_desc=''),
 'Q90': Entity(qid='Q90', ner_labels=['LOC'], mention_frequency=3, unique_surface_forms=['Paris'], short_desc=''),
 'Q1405281': Entity(qid='Q1405281', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['Troie'], short_desc=''),
 'Q142': Entity(qid='Q142', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['France'], short_desc=''),
 'Q1469': Entity(qid='Q1469', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['Loire'], short_desc=''),
 'Q193821': Entity(qid='Q193821', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['Saumur'], short_desc=''),
 'Q1471': Entity(qid='Q1471', ner_labels=['LOC'], mention_frequency=1, unique_surface_forms=['Seine'], short_desc='')}

In [11]:
# use the  SalientSentenceSelector to extract the top 5 sentences for the most frequent person in the document
# this entity context made of the k sentences can then be fed to an LLM (vel sim.) to characterise the spation-temporal
# dimension of the document
person_context = sass.select(top_k_sentences=5, entity_type='person')

In [12]:
person_context

(Entity(qid='Q18190448', ner_labels=['PER'], mention_frequency=52, unique_surface_forms=['Guillaume'], short_desc=''),
 [Guillaume Que c'est un grapd trésor que notre liberté, Qu'on la compare mal au bien de la clartète, Que l'usage en est doux, et qu'au temps où nous sommes, Un bien qui vient du Ciel se vend mal à des hommes, Le libéral arbitre est un don précieux Par où nous éprouvons la clémence des Dieux, Un esprit franc et libre où la raison abonde Enne possédant rien, possède tout le monde, Il ne dépend jamais de tant d'esprits divers, Au lieu d'une maison il a tour d'Unissers, Sa richesse le suit, et l'âme, sans contrainte, Sans auoir ai vertu sans con visage empreinte, LES PÂTONS,
  Quand elle aura connu mon âme par ta bouche, Regarde dans son teint comment cela la touche, Prends garde si l'amour retient ensevelis Sous un peu de rougeur, ses roses et ses lis, Compte tous ses soupirs, et te donne la gloire D'âpporter mon salut dans ta belle mémoire, Guillaume Tout ce que l'arti

In [13]:
# use the  SalientSentenceSelector to extract the top 5 sentences for the most frequent person in the document
# this entity context made of the k sentences can then be fed to an LLM (vel sim.) to characterise the spation-temporal
# dimension of the document
place_context = sass.select(top_k_sentences=5, entity_type='place')

In [20]:
place_context

(Entity(qid='Q90', ner_labels=['LOC'], mention_frequency=3, unique_surface_forms=['Paris'], short_desc=''),
 [Guillaume Ah, mon ami, qu'aimer est une belle chose, Si vous saviez l'état où votre vis maintenant, Je ne fais plus de vœux à Carême-prenanf, Bacchus m'est ennemi, je fais geler les treilles, J'ai hanni les festins, j'ai rompu les bouteilles, Je ne vis que d'amour, de cipre, et d'ambregris, Je pense que le Ciel est tombé dans Paris, Ne parlez plus de vous tant de vos Artenices, On n'a jamais rien venger d'égal à mes supplices, Comme elle vous trahit en quittant ce seiour.,
  tu me désespère Dans les délices de Paris, Où ie suis touché de Cloris Comme un roche d'H vipère, Toutes les nuits rêvant à toi Je t'appelle mon périt Roi, Je parle avec ton image, Et te fais croire à mes desseins Que le Ciel approuve l'hommage,
  Paris en fit de même, et son Heleine aussi Souffrait innocemment par un même souci, Mon amour y consent, qu'est-ce, bien qu'un supplice Quand il est question de s

In [ ]:
# Guillaume is the most frequent entity in the document
# just because Calyante was not linked to a Wikidata QID
# To be dealt with and fixed in the next iteration
sass._mentions_df.mention.value_counts()

mention
Calyante     70
Guillaume    54
Filandre     39
Agaritte     37
Aronthe      37
             ..
Alexandre     1
Nectar        1
Vulcain       1
Jupiter       1
Ciela         1
Name: count, Length: 133, dtype: int64

In [ ]:
# the summary should contain:
# document metadata: author, title, publication date
# top 5 person mentions
# top 5 place mentions
# most salient person entity + top 5 sentences
# most salient place entity + top 5 sentences
def build_JSON_document_summary():
    pass